In [1]:
import asyncio
import os

from typing import Iterable

import pycoq.opam
import pycoq.common
import pycoq.agent

In [62]:
coq_ctxt = pycoq.common.CoqContext(pwd=os.getcwd(),
                                   executable='',
                                   target='serapi_shell')
cfg = pycoq.opam.opam_serapi_cfg(coq_ctxt)

In [85]:
coq = pycoq.serapi.CoqSerapi(cfg)

In [55]:
theorems = [
    ("Theorem add_easy: forall n:nat, 0 + n = n.",
     ["intros.", "Show Proof.", "simpl.", "Show Proof.", "reflexivity.", "Show Proof."]),
]
prop, script = theorems[0]

In [86]:
await coq.start()
coq

In [87]:
cmd_tag, resp_ind, coq_exc, sid = await coq.execute(prop)
cmd_tag, resp_ind, coq_exc, sid

(1, 23, [], [2])

In [78]:
async def query_proof(coq) -> str:
    """
    sends serapi command
    (Query () Definition name)
    """
    cmd_tag = len(coq._sent_history)
    cmd = f'(Query (pp: pp_format: PpStr) Proof)'
    await coq._kernel.writeline(cmd)
    print('writen')
    coq._sent_history.append(cmd)
    return cmd_tag

async def query_proof_completed(coq):
    cmd_tag = await query_proof(coq)
    print('queries')
    resp_ind = await coq.wait_for_answer_completed(cmd_tag)
    print('waited')
    coqexns = await coq.coqexns(cmd_tag)
    print('exns')
    if coqexns != []:
        raise RuntimeError(f'Unexpected error during coq-serapi command Query () Goals '
                           f'with CoqExns {coqexns}')
    goals = await coq._answer(cmd_tag)
    return goals

In [88]:
for stmt in script:
    print(f"executing: {stmt}")
    _, _, coq_exc, _ = await coq.execute(stmt)

    if coq_exc:
        print(f"evaluation of {stmt} in coq-serapi session raised exception {coq_exc}")
        break

    serapi_goals = await coq.serapi_goals()
    serapi_proofs = await query_proof_completed(coq)
    print(serapi_proofs)

executing: intros.
writen
queries


CancelledError: 